# Análise de Sentimento de Reviews (MVP - Semana 2)

Este notebook cumpre os requisitos da Semana 2 do projeto de Big Data. O objetivo é carregar os reviews de produtos coletados na Semana 1, realizar a limpeza e o pré-processamento do texto, aplicar um modelo de Processamento de Linguagem Natural (NLP) para classificar o sentimento de cada review e, por fim, salvar o resultado em um novo arquivo CSV.

**Processo:**
1.  **Carregamento dos Dados:** Leitura do arquivo `comentarios_produtos.csv`.
2.  **Limpeza e Pré-processamento:** Normalização do texto para remover ruídos.
3.  **Análise de Sentimento:** Aplicação de um modelo híbrido para classificação.
4.  **Salvamento:** Gravação do resultado final.

### Passo 0: Instalação de Dependências

Esta célula instala as bibliotecas Python necessárias para o notebook.

* `leia-br`: Biblioteca para análise de sentimento em português.
* `unidecode`: Ferramenta para normalização de texto e remoção de acentos.

Após a instalação, o kernel do Python é reiniciado automaticamente para carregar as novas bibliotecas.

In [23]:
# Instala as bibliotecas necessárias para a análise
%pip install leia-br unidecode

# Reinicia o kernel do Python para que as bibliotecas instaladas fiquem disponíveis
# dbutils.library.restartPython()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Passo 1: Setup e Carregamento dos Dados

A primeira etapa do nosso processo ETL (Extração, Transformação, Carga) é a **Extração**. Nesta fase, vamos importar as bibliotecas Python que serão utilizadas e carregar os dados brutos de comentários que foram coletados na Semana 1.

O arquivo `comentarios_produtos.csv`, armazenado em um Volume do Databricks, será lido e carregado em um DataFrame da biblioteca `pandas`, que é a estrutura fundamental sobre a qual faremos todas as nossas manipulações.

In [24]:
# Passo 1: Setup e Carregamento dos Dados

import pandas as pd
import sys
import os

# Pega o caminho do diretório onde o notebook está (a pasta 'notebooks/')
try:
    # Esta linha funciona na maioria dos ambientes de script
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
except NameError:
    # Este é o fallback que funciona em notebooks interativos (Jupyter, Databricks)
    notebook_dir = os.getcwd()

# Navega um nível acima ('../') para chegar à pasta raiz do projeto ('bi-web-scrapping/')
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Cria o caminho relativo para a pasta 'src/'
src_path = os.path.join(project_root, 'src')

# Adiciona a pasta 'src' ao path do sistema para permitir as importações
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Agora, a importação funcionará de forma relativa ao projeto
from processing.text_processor import limpar_texto, classificar_sentimento
print("Módulo 'text_processor' importado com sucesso!")


# --- Carregamento do Arquivo com Caminho Relativo ---

# Constrói o caminho relativo para o arquivo de dados
file_path = os.path.join(project_root, 'data', 'raw', 'comentarios_produtos.csv')

try:
    df_reviews = pd.read_csv(file_path, sep='|', on_bad_lines='skip')
    print("Arquivo de comentários carregado com sucesso!")
    display(df_reviews.head())
except FileNotFoundError:
    print(f"\nERRO: Arquivo não encontrado!")
    print("Por favor, verifique se a sua estrutura de pastas está assim:")
    print("bi-web-scrapping/ -> data/ -> raw/ -> comentarios_produtos.csv")
except Exception as e:
    print(f"\nOcorreu um erro ao carregar o arquivo: {e}")

Módulo 'text_processor' importado com sucesso!
Arquivo de comentários carregado com sucesso!


,id_produto,id_avaliacoes,id_comentario,rating_do_comentario,quantidade_avaliacoes_do_produto,comentario_about_produto,data_coleta
0,80054742,80054742,68ea88d327ae04a3a6680c51,5,2308,O smartphone parece ser muito bom apesar de ai...,2025-10-22 21:16:16
1,80054742,80054742,68ac9172bbb755975d29447a,5,2308,Achei uma ótima escolha. Muito útil para traba...,2025-10-22 21:16:16
2,80054742,80054742,68b60274312495cfd879bd61,5,2308,Muito bom e entrega super rápida no mesmo dia.,2025-10-22 21:16:16
3,80054742,80054742,68ac9190bbb755975d29ab31,5,2308,Excelente produto.,2025-10-22 21:16:16
4,80054742,80054742,68ac9172bbb755975d293822,4,2308,Ótimo,2025-10-22 21:16:16


## Passo 2: Limpeza e Pré-processamento do Texto

A limpeza dos dados textuais é uma etapa fundamental para garantir a qualidade da análise. Um texto com ruídos (pontuação, letras maiúsculas, caracteres especiais) pode confundir o modelo de NLP.

Nesta etapa, vamos:
1.  Remover comentários vazios.
2.  Converter todo o texto para minúsculas.
3.  Normalizar os caracteres, removendo acentos (ex: `ótimo` vira `otimo`).
4.  Remover tudo que não for letras ou espaços.

Isso cria uma nova coluna, `comentario_limpo`, que servirá de base para a nossa análise.

In [25]:
df_cleaned = df_reviews.copy()
df_cleaned.dropna(subset=['comentario_about_produto'], inplace=True)

# Aplica a função de limpeza importada do nosso módulo
df_cleaned['comentario_limpo'] = df_cleaned['comentario_about_produto'].apply(limpar_texto)

df_cleaned = df_cleaned[df_cleaned['comentario_limpo'].str.strip() != '']

print("Limpeza de texto concluída.")
display(df_cleaned[['comentario_about_produto', 'comentario_limpo']].head())

Limpeza de texto concluída.


,comentario_about_produto,comentario_limpo
0,O smartphone parece ser muito bom apesar de ai...,o smartphone parece ser muito bom apesar de ai...
1,Achei uma ótima escolha. Muito útil para traba...,achei uma otima escolha muito util para trabal...
2,Muito bom e entrega super rápida no mesmo dia.,muito bom e entrega super rapida no mesmo dia
3,Excelente produto.,excelente produto
4,Ótimo,otimo


## Passo 3: Classificação de Sentimento com Abordagem Híbrida

A análise inicial mostrou que usar apenas um modelo de texto ou apenas a nota do usuário gera inconsistências. Para obter um resultado mais preciso e profissional, implementamos uma **abordagem híbrida de múltiplas camadas**:

1.  **Camada 1 (Regras por Nota):** A nota do usuário (`rating`) é a fonte mais confiável. Definimos que:
    * **4 ou 5 estrelas** são classificadas como `Positivo`.
    * **1 ou 2 estrelas** são classificadas como `Negativo`.

2.  **Camada 2 (NLP para Ambiguidade):** Comentários com **3 estrelas** são ambíguos. Nesses casos, usamos a biblioteca `LeIA` para analisar o texto e definir o sentimento.

3.  **Camada 3 (Correção de Inconsistências):** O principal desafio são os usuários que dão uma nota baixa (1 ou 2) mas escrevem um texto positivo (ex: "Ótimo"). Nossa camada final corrige isso:
    * Ela reavalia todos os comentários classificados como `Negativo`.
    * Se o texto contiver palavras-chave fortemente positivas (como "ótimo", "perfeito", "adorei") ou se a análise do `LeIA` mostrar um sentimento positivo, a classificação é **corrigida** para `Positivo`.

In [26]:
print("Iniciando a classificação de sentimento com o módulo de processamento...")

# Aplica a função de classificação completa importada do nosso módulo
df_final = classificar_sentimento(df_cleaned)

print("\nAnálise de sentimento concluída!")
display(df_final['sentimento'].value_counts())

Iniciando a classificação de sentimento com o módulo de processamento...

Análise de sentimento concluída!


sentimento
Positivo    2162
Negativo      43
Neutro         5
Name: count, dtype: int64

## Passo 4: Validação e Salvamento do Resultado Final

Com a classificação concluída, a última etapa é gerar o arquivo CSV final, que servirá como entregável para a Semana 2 e como insumo para as próximas fases do projeto. O arquivo será salvo como `comentarios_classificados.csv` no mesmo Volume.

In [28]:
# Passo 4: Validação e Salvamento do Resultado Final

# Validação: exibe os primeiros 20 registros para uma última verificação
print("Exibindo os primeiros 20 registros do DataFrame final para validação:")
display(df_cleaned[['rating_do_comentario', 'sentimento', 'comentario_about_produto']].head(20))

# O arquivo será salvo em 'bi-web-scrapping/data/output/comentarios_classificados.csv'
output_path_absolute = os.path.join(project_root, 'data', 'output', 'comentarios_classificados.csv')

# Cria uma versão do caminho relativa à pasta do projeto para exibição
output_path_relative = os.path.relpath(output_path_absolute, project_root)

# Substitui as barras invertidas por barras normais para um visual mais limpo no print
output_path_display = output_path_relative.replace('\\\\', '/')

# Seleciona e ordena as colunas para o arquivo final
colunas_para_salvar = [
    'id_produto', 'id_avaliacoes', 'id_comentario', 'rating_do_comentario',
    'comentario_about_produto', 'sentimento', 'data_coleta'
]
df_final = df_cleaned[colunas_para_salvar]

# Salva o DataFrame em um novo arquivo CSV, usando '|' como separador
try:
    df_final.to_csv(output_path_absolute, sep='|', index=False, encoding='utf-8')
    print(f"\nSucesso! O arquivo com os sentimentos classificados foi salvo em:\n{output_path_display}")
except Exception as e:
    print(f"\nOcorreu um erro ao salvar o arquivo: {e}")

Exibindo os primeiros 20 registros do DataFrame final para validação:


,rating_do_comentario,sentimento,comentario_about_produto
0,5,Positivo,O smartphone parece ser muito bom apesar de ai...
1,5,Positivo,Achei uma ótima escolha. Muito útil para traba...
2,5,Positivo,Muito bom e entrega super rápida no mesmo dia.
3,5,Positivo,Excelente produto.
4,4,Positivo,Ótimo
5,5,Positivo,Muito bom
6,5,Positivo,Samsung é uma ótima marca de qualidade e eu se...
7,4,Positivo,O produto é bom
8,5,Positivo,"Excelente produto, comprei para minha mãe e el..."
9,5,Positivo,"Chegou adiantado, ele é muito lindo, boa image..."



Sucesso! O arquivo com os sentimentos classificados foi salvo em:
data\output\comentarios_classificados.csv
